<a href="https://colab.research.google.com/github/hikaru677/Frame-interpolation-dataset-creation/blob/main/all_frame_vimeo_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import random
import os
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import tempfile
import math
from tqdm.notebook import tqdm
import copy

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! cd /content/drive

In [ ]:
! pip install torch
! pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install openmim
!mim install mmcv-full==1.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html


In [ ]:
! pip install mmflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mim download mmflow --config pwcnet_ft_4x1_300k_sintel_final_384x768

processing pwcnet_ft_4x1_300k_sintel_final_384x768...
pwcnet_ft_4x1_300k_sintel_final_384x768.pth exists in /root/.cache/mim
Successfully dumped pwcnet_ft_4x1_300k_sintel_final_384x768.py to /root/.cache/mim


In [ ]:
from mmflow.apis import init_model, inference_model
from mmflow.datasets import visualize_flow, write_flow
import mmcv
config_file = '/root/.cache/mim/pwcnet_ft_4x1_300k_sintel_final_384x768.py'
checkpoint_file = '/root/.cache/mim/pwcnet_ft_4x1_300k_sintel_final_384x768.pth'

# estimate

In [ ]:
# fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
unverified_context = ssl._create_unverified_context()
UCF_DATA_FOLDER = "/content/drive/MyDrive/UCF101_Unzipped_Files/UCF-101"

def list_ucf_videos():
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi)", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

# def fetch_ucf_video(video):
#     action_dir = video.split('_')[1]
#     if action_dir == "HandstandPushups":
#       action_dir = "HandStandPushups"
#       text_list = video.split('_')
#       base_video_path = text_list[0]+ '_' + action_dir + '_' + text_list[2] + '_' + text_list[3]
#       video_path = os.path.join(UCF_DATA_FOLDER,action_dir,base_video_path)
#       print(video_path)
#       return video_path

#     video_path = os.path.join(UCF_DATA_FOLDER,action_dir,video)
#     return video_path

def path_modify(video):
  action_dir = video.split('_')[1]
  if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
      text_list = video.split('_')
      base_video_path = text_list[0]+ '_' + action_dir + '_' + text_list[2] + '_' + text_list[3]
      print(base_video_path)
      return base_video_path

  return video


In [ ]:
def fetch_action_list(action = "BandMarching"):
    list_data = list_ucf_videos()
    return [i for i in list_data if i.split('_')[1] == action]

In [ ]:
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

#
img1='/content/drive/MyDrive/UCF101_every_frame/Fixed/Archery/frames_v_Archery_g02_c01/000002.png'
img2='/content/drive/MyDrive/UCF101_every_frame/Fixed/Archery/frames_v_Archery_g02_c01/000003.png'
img3='/content/drive/MyDrive/UCF101_every_frame/Fixed/Archery/frames_v_Archery_g02_c01/000004.png'

# img1 = "/content/drive/MyDrive/middleburry_8_frame/Walking/frame10.png"
# img2 = "/content/drive/MyDrive/middleburry_8_frame/Walking/frame11.png"
# img3 = "/content/drive/MyDrive/middleburry_8_frame/Walking/frame12.png"

load checkpoint from local path: /root/.cache/mim/pwcnet_ft_4x1_300k_sintel_final_384x768.pth


In [ ]:
def flo_out(model,img1,img2,img3):
  result_12 = inference_model(model, img1, img2)
  result_23 = inference_model(model, img2, img3)

  write_flow(result_12,flow_file = 'flow_12.flo')
  write_flow(result_23,flow_file = 'flow_23.flo')

  visualize_flow(result_12, save_file='flow_map_12.png')
  # visualize_flow(-result_12,save_file = 'flow_map_reverse.png')
  visualize_flow(result_23,save_file = "flow_map_23.png")
  return 'flow_12.flo','flow_23.flo'

In [ ]:
def remove_values_closeto_0(flow_map12,flow_map23):
  condition_12_1 = (flow_map12 < 0.01)
  condition_12_2 = (flow_map12 > -0.01)
  condition_12 = condition_12_1 & condition_12_2

  condition_23_1 = (flow_map23 < 0.01)
  condition_23_2 = (flow_map23 > 0.01)
  condition_23 = condition_23_1 & condition_23_2

  flow_map12[condition_12] = 0
  flow_map23[condition_23] = 0

  return flow_map12,flow_map23


def read_2_flo(flow12_path,flow23_path):
  flow_map12 = cv2.readOpticalFlow(flow12_path)
  flow_map23 = cv2.readOpticalFlow(flow23_path)

  flow_map12,flow_map23 = remove_values_closeto_0(flow_map12,flow_map23)


  return flow_map12,flow_map23

In [ ]:
def convert_nan_to_0(flow_magnitude):
  nans = np.isnan(flow_magnitude)
  if np.any(nans):
      nans = np.where(nans)
      flow_magnitude[nans] = 0.

  return flow_magnitude


def flow_magnitude_out(flow_map12,flow_map23):
  flow_map_12_channel_0 = flow_map12[:,:,0]
  flow_map_12_channel_1 = flow_map12[:,:,1]

  flow_map_23_channel_0 = flow_map23[:,:,0]
  flow_map_23_channel_1 = flow_map23[:,:,1]

  flow_magnitude_12, _ = cv2.cartToPolar(flow_map_12_channel_0, flow_map_12_channel_1)
  flow_magnitude_23, _ = cv2.cartToPolar(flow_map_23_channel_0, flow_map_23_channel_1)
  # A couple times, we've gotten NaNs out of the above...

  return flow_magnitude_12,flow_magnitude_23


In [ ]:
flo1 ,flo2 = flo_out(model,img1,img2,img3)
flow_map12 ,flow_map23 = read_2_flo(flo1,flo2)
flow_magnitude_12,flow_magnitude_23 = flow_magnitude_out(flow_map12,flow_map23)

In [ ]:
def warp_second_frame_to_first(flow_map_origin,second_frame):
  flow_map = copy.copy(flow_map_origin)
  h,w = flow_map.shape[:2]
  flow_map[:,:,0] += np.arange(w)#変換後の座標を指定
  flow_map[:,:,1] += np.arange(h)[:,np.newaxis]
  img2 = cv2.imread(second_frame)
  warped_prevImg = cv2.remap(img2, flow_map, None, cv2.INTER_LINEAR)#INTER_NEAREST#INTER_AREA
  cv2.imwrite("warped_first_frame.png",warped_prevImg)

  return warped_prevImg

In [ ]:
print(flow_map12)

In [ ]:
# flow = flow_map12
# h, w = flow.shape[:2]
# flow[:,:,0] += np.arange(w)#変換後の座標を指定
# flow[:,:,1] += np.arange(h)[:,np.newaxis]
# img_ = cv2.imread(img2)
# prevImg = cv2.remap(img_, flow, None, cv2.INTER_LINEAR)#INTER_NEAREST#INTER_AREA
# cv2.imwrite("Image.png",prevImg)

In [ ]:
prevImg = warp_second_frame_to_first(flow_map12,img2)
img_ = cv2.imread(img2)

In [ ]:
print(flow_map12)

[[[-0.09586643  0.        ]
  [-0.09586643  0.        ]
  [-0.08887243  0.        ]
  ...
  [-0.36849964  0.05834742]
  [-0.39069784  0.06832174]
  [-0.39069784  0.06832174]]

 [[-0.09544005  0.        ]
  [-0.09544005  0.        ]
  [-0.08843189 -0.01085257]
  ...
  [-0.36602268  0.05774394]
  [-0.38796917  0.06762376]
  [-0.38796917  0.06762376]]

 [[-0.09018145 -0.01581354]
  [-0.09018145 -0.01581354]
  [-0.08299853 -0.02811566]
  ...
  [-0.33547348  0.05030102]
  [-0.3543152   0.05901538]
  [-0.3543152   0.05901538]]

 ...

 [[-0.10097285  0.12747262]
  [-0.10097285  0.12747262]
  [-0.09964161  0.12887283]
  ...
  [-0.19900411 -0.03155718]
  [-0.19028392 -0.033509  ]
  [-0.19028392 -0.033509  ]]

 [[-0.10197476  0.12944153]
  [-0.10197476  0.12944153]
  [-0.10161703  0.13100374]
  ...
  [-0.18525873 -0.03080775]
  [-0.17405327 -0.03221573]
  [-0.17405327 -0.03221573]]

 [[-0.102056    0.12960118]
  [-0.102056    0.12960118]
  [-0.10177721  0.13117653]
  ...
  [-0.18414417 -0.030746

## Vimeo's three conditions

In [ ]:
def with_movement_greater_than_3(flow_map,flow_magnitude):#2
  first_bool = False
  more_number = np.count_nonzero(flow_magnitude > 3)
  pixel_number = flow_map.shape[0] * flow_map.shape[1]# 240*320=76800
  print(more_number)
  print(pixel_number)
  print(more_number/pixel_number * 100)
  if (more_number/pixel_number * 100) > 5:
    first_bool = True

  return first_bool

In [ ]:
with_movement_greater_than_3(flow_map12,flow_magnitude_12)

7759
76800
10.102864583333334


True

In [ ]:
def intensity_levels(warped_prevImg,prevImg):#4
  second_bool = False
  img_diff = cv2.absdiff(warped_prevImg, prevImg)
  print(img_diff.shape)
  sum_img_diff = np.sum(img_diff,axis = 2)
  print(sum_img_diff.shape)
  cv2.imwrite('sabun-result.jpg',img_diff)
  average_l1_distance = np.mean(sum_img_diff)
  print(average_l1_distance)
  if average_l1_distance < 15:
    second_bool = True

  return second_bool

In [ ]:
intensity_levels(prevImg,img_)

(240, 320, 3)
(240, 320)
23.753424479166668


False

In [ ]:
def average_difference_v21_v23(flow_map12 ,flow_map23):#5
  third_bool = False

  flow_map_diff = flow_map23 - flow_map12
  print(flow_map23)
  print("---------------------")
  print(flow_map12)
  # print("---------------------")
  # print(flow_map_diff)
  # print("---------------------")
  print(flow_map_diff.shape)
  # print("---------------------")
  flow_map_diff_c1 = flow_map_diff[:,:,0]
  flow_map_diff_c2 = flow_map_diff[:,:,1]
  flow_map_disagreement, _ = cv2.cartToPolar(flow_map_diff_c1, flow_map_diff_c2)
  # print(flow_map_disagreement)
  # print("---------------------")
  # print(flow_map_disagreement)
  flow_map_mean_disagreement = np.mean(flow_map_disagreement)
  print(flow_map_mean_disagreement)
  # print(np.sqrt((flow_map_diff_c1[0][0])**2 + (flow_map_diff_c2[0][0])**2))

  if flow_map_mean_disagreement < 1.0:
    third_bool = True
  return third_bool

In [ ]:
average_difference_v21_v23(flow_map12 ,flow_map23)

[[[-0.07714942  0.02252454]
  [-0.07714942  0.02252454]
  [-0.06345761  0.02670406]
  ...
  [-0.32886848  0.02857471]
  [-0.35179412  0.03680546]
  [-0.35179412  0.03680546]]

 [[-0.07739115  0.02228117]
  [-0.07739115  0.02228117]
  [-0.06368165  0.02640355]
  ...
  [-0.32637045  0.02821669]
  [-0.34904101  0.03640472]
  [-0.34904101  0.03640472]]

 [[-0.0803724   0.01927951]
  [-0.0803724   0.01927951]
  [-0.06644491  0.02269725]
  ...
  [-0.29556155  0.02380112]
  [-0.3150857   0.03146226]
  [-0.3150857   0.03146226]]

 ...

 [[-0.07686289  0.14478473]
  [-0.07686289  0.14478473]
  [-0.07477099  0.13964178]
  ...
  [-0.11383876 -0.08871308]
  [-0.10434688 -0.0951651 ]
  [-0.10434688 -0.0951651 ]]

 [[-0.0731561   0.14776152]
  [-0.0731561   0.14776152]
  [-0.07189345  0.14263992]
  ...
  [-0.09337433 -0.08455703]
  [-0.0813156  -0.09117381]
  [-0.0813156  -0.09117381]]

 [[-0.07285553  0.14800291]
  [-0.07285553  0.14800291]
  [-0.07166012  0.14288303]
  ...
  [-0.09171496 -0.084220

False

In [ ]:
def fetch_ucf_action_list():
  cache_list = []
  video_list = list_ucf_videos()
  for path in video_list:
    # base_path = path.split('/')[-1]
    action_dir = path.split('_')[1]
    if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
    cache_list.append(action_dir)
  cache_list = set(cache_list)
  cache_list = list(cache_list)
  cache_list.sort()
  return cache_list

In [ ]:
def Fixed_list_ucf(classify = "Fixed",div = 0):
    cache_list = []
    action_list = fetch_ucf_action_list()
    # if div == 0:
    #     action_list = action_list[0:50]
    # else:
    #     action_list = action_list[50:101]
    classify_dir = os.path.join("/content/drive/MyDrive/UCF101_every_frame",classify)
    for action in action_list:
        action_path = os.path.join(classify_dir,action)
        each_frame_folder_list = os.listdir(action_path)
        cache_list.append(each_frame_folder_list)
    return np.hstack(cache_list)

In [ ]:
def frame_folder_full_path(folder_name,classify):#ex:file_name = v_ApplyEyeMakeup_g01_c01
  #ex:folder_name = frames_v_BenchPress_g24_c04
  action_dir = folder_name.split('_')[2]
  folder_name = os.path.join("/content/drive/MyDrive/UCF101_every_frame/",classify,action_dir,folder_name)
  # folder_name = "/content/drive/MyDrive/UCF101_every_frame/" + classify + "/"+ action_dir + folder_name
  print(folder_name)
  return folder_name

In [ ]:
def choice_three_frame(frame_folder_path):#ex:frame_folder_path = /content/drive/MyDrive/UCF101_every_frame/Fixed/BlowDryHair/frames_v_BlowDryHair_g01_c01
  count = 0
  triplet_frame_count = 0

  frames_list = os.listdir(frame_folder_path)
  sorted(frame_list)
  pirnt(frame_list)
  last_frame_num = frame_list[-1]
  print(last_frame_num)
  last_frame_num = os.path.splitext(last_frame_num)[0]

  #Set the first 3 frames
  img1 = os.path.join(frame_folder_path, str(count).zfill(6)+ ".png")
  print(img1)
  img2 = os.path.join(frame_folder_path, str(count+1).zfill(6)+ ".png")
  print(img2)
  img3 = os.path.join(frame_folder_path, str(count+2).zfill(6)+ ".png")
  print(img3)

  while True:
    flo1 ,flo2 = flo_out(model,img1,img2,img3)
    flow_map12 ,flow_map23 = read_2_flo(flo1,flo2)
    flow_magnitude_12,flow_magnitude_23 = flow_magnitude_out(flow_map12,flow_map23)

    first_bool_12 = with_movement_greater_than_3(flow_map,flow_magnitude)
    first_bool_23 = with_movement_greater_than_3(flow_map,flow_magnitude)

    warped_prevImg2_to_1 = warp_second_frame_to_first(flow_map12,img2)
    warped_prevImg3_to_2 = warp_second_frame_to_first(flow_map23,img3)
    second_bool_12 = intensity_levels(warped_prevImg,prevImg)
    second_bool_23 = intensity_levels(warped_prevImg,prevImg)

    third_bool = average_difference_v21_v23(flow_map12 ,flow_map23)

    if (first_bool_12 and first_bool_23) and (second_bool_12 and second_bool_23) and (third_bool):
      #Trueなら3frameずらしてtripletのカウントを+1
      os.makedirs("")

In [ ]:
choice_three_frame("/content/drive/MyDrive/UCF101_every_frame/Fixed/BlowDryHair/frames_v_BlowDryHair_g01_c01")

In [ ]:
classify = "Fixed" #Unfixed#Fixed
fetch_frame_folder_list = Fixed_list_ucf()#Fixedの中からframeをfetchしてこなければならない
#↑これでframe_v_～～のfolderのlistまでは取り出せる
for fetch_path in tqdm(fetch_frame_folder_list):
  frame_folder = frame_folder_full_path(fetch_path,classify)


  0%|          | 0/5717 [00:00<?, ?it/s]

ストリーミング出力は最後の 5000 行に切り捨てられました。
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g24_c04
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g24_c06
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g25_c01
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g25_c02
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g25_c03
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g25_c04
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g25_c05
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g25_c06
/content/drive/MyDrive/UCF101_every_frame/Fixed/BenchPress/frames_v_BenchPress_g25_c07
/content/drive/MyDrive/UCF101_every_frame/Fixed/Biking/frames_v_Biking_g03_c01
/content/drive/MyDrive/UCF101_every_frame/Fixed/Biking/frames_v_Biking_g03_c04
/content/drive/MyDrive/UCF1